In [190]:
# Iván Andrés Trujillo Abella
# ivantrujillo1229@gmail.com

In [191]:
# First we want
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/it-ces/Datasets/main/basesuper.csv")

In [192]:
X = ['Ingresos de actividades ordinarias','Costo de ventas',
      'Ganancia (pérdida) antes de impuestos',
     'Total de activos', 'Total pasivos',
      'Patrimonio total', 'Ganancia (pérdida) por actividades de operación',
      'Pasivos corrientes totales',  'Ganancia bruta',
       'Ganancia (pérdida)', 'Ganancias acumuladas', 'y' ]
df  = df[X]

In [193]:
df.dropna(inplace=True)

In [194]:
# how many models you can test?
2**(df.shape[1]-1)

2048

In [195]:
def Xy(df_,target):
    """
    Split the data in X,y to ML implementations
    """
    df = df_.copy()
    X = df.loc[ : , df.columns != target]
    y = df[target]
    return X,y

In [196]:
X,y = Xy(df, 'y')
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X, y = rus.fit_resample(X, y)

In [197]:
## Testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle = True, random_state = 666, stratify=y)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.30, shuffle = True, random_state = 666, stratify=y_train)

In [198]:
print(y_train.value_counts())
print(y_test.value_counts())
print(y_val.value_counts())

0    109
1    109
Name: y, dtype: int64
0    47
1    47
Name: y, dtype: int64
1    33
0    33
Name: y, dtype: int64


In [199]:
## Write the guided function
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
def score(solution, X_train_, y_train_):
  X_train, y_train = X_train_.copy(), y_train_.copy()
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.30, shuffle = True,
                                                    random_state = 666, stratify=y_train)
  solution = np.array(solution, dtype='bool')
  X_train = X_train.loc[:, solution].to_numpy()
  X_val = X_val.loc[:, solution].to_numpy()
  return f1_score(y_val, LogisticRegression().fit(X_train, y_train).predict(X_val)),


In [200]:
sol = [1,1,0,0,1,0,1,1,1,1,0]
score(sol, X_train, y_train)

(0.746268656716418,)

In [201]:
!pip install deap

In [202]:
import random
from deap import creator
from deap import base
from deap import tools
from deap import algorithms
import matplotlib.pyplot as plt

In [203]:
chromosomal_size = X_train.shape[1]

In [204]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("binaryGen", random.randint, 0,1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binaryGen, chromosomal_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, 80)
toolbox.register("evaluate", lambda x: score(x,X_train, y_train))
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3) #change to rolloute wheel
pop = toolbox.population()
pop, logbok = algorithms.eaSimple(pop, toolbox, cxpb=0.35, mutpb=0.05, ngen=60, verbose=False)
print(pop[0])
# Could be very useful graph while is running to see the behavior of the algorithm!!!
# uses method fit to any model of scikit-learn

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


[0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]


In [209]:
pop[0]

[0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]

In [208]:
toolbox.evaluate(pop[0])

(0.8,)

In [207]:
## Add graph

In [213]:
X_train.loc[:,np.array(pop[0], dtype='bool')]

,Total pasivos,Ganancia (pérdida) por actividades de operación,Ganancia bruta,Ganancia (pérdida)
81,52081455.0,1433558.0,1934539.0,1255619.0
155,5431759.0,3037631.0,5302559.0,1950280.0
221,2462816.0,596682.0,1973692.0,399777.0
222,7054691.0,-2266002.0,6106411.0,-2266002.0
163,10358025.0,-204624.0,174359.0,-676806.0
...,...,...,...,...
249,12693877.0,2416663.0,7282917.0,596802.0
96,1334277.0,-17268.0,0.0,-82815.0
7,10672339.0,1470879.0,4795260.0,970638.0
116,14356612.0,2322565.0,5694295.0,1166834.0
